In [1]:
import os
for dirname, _, filenames in os.walk('./dataset/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./dataset/.~lock.train.csv#
./dataset/raw_data.csv
./dataset/sample_submission.csv
./dataset/test.csv
./dataset/train.csv
./dataset/train_data_dict.csv


In [2]:
import numpy as np
import pandas as pd 
import os
import warnings
warnings.filterwarnings('ignore')
from collections import Counter
from sklearn.model_selection import train_test_split

In [13]:
df      = pd.read_csv('./dataset/train.csv')
train_desc_df = pd.read_csv('./dataset/raw_data.csv')

In [4]:
# cleaning
df = df.drop(['case_id','patientid'],axis=1)
df = df.dropna(axis=0)
# spliting
train_df , test_df  = train_test_split(df, test_size=0.2)

# conveting to list 
#data1 = train_df.values.tolist()
#train_df.head()

In [5]:
# get the deffirant keys (columns names)
keys = train_df.columns

In [6]:
def discritisation(listA , listB , n):
    listt = []
    for i in range(len(listA)):
        listt.append((listA[i], listB[i]))
    new_list = sorted(listt)
    # Step-1 : Pre-division into as many intervals as there is a class change. Separator //
    # Step-2 : If we have same value X in different intervals, we remove the separators between these intervals
    # to have the values of X in the same interval
    i=0
    while i< len(new_list)-1 :
        if new_list[i][1] != new_list[i+1][1]:
            if new_list[i][0] != new_list[i+1][0]:
                new_list.insert(i+1, '//')
                i += 2
            else :
                i += 1
        else :
            i += 1
    # Step-3 : Grouping of intervals such that there are at least n values of the same class in each interval
    i = 0
    class_cpt = {'0-10': 0,'11-20': 0,'21-30': 0,'31-40': 0,'41-50': 0,'51-60': 0,
             '61-70': 0,'71-80': 0,'81-90': 0,'91-100': 0,'More than 100 Days': 0}
    while i < len(new_list) - 1 :
        while new_list[i] != '//' and i < len(new_list) - 1  :
            class_cpt[new_list[i][1]] +=1
            i += 1
        flagN = 0
        for valeur in class_cpt.values():
            if valeur >= n :
                flagN=1
        if (not flagN):
            new_list.pop(i)
        else :
            i += 1
            class_cpt = {'0-10': 0,'11-20': 0,'21-30': 0,'31-40': 0,'41-50': 0,'51-60': 0,
             '61-70': 0,'71-80': 0,'81-90': 0,'91-100': 0,'More than 100 Days': 0}
     
    # Step-4 : Grouping of adjacent intervals gaving the same majority clas
    i = 0
    posToDelete = 0
    majClass0 = ''
    majValue = 0
    resultList = []
    while i <= len(new_list) - 1 :
        while new_list[i] != '//' and i < len(new_list) - 1 :
            class_cpt[new_list[i][1]] +=1
            i += 1
        
        max_value = max(class_cpt, key=class_cpt.get)
        if majClass0 != max_value:
            if majClass0 != '' :
                mean = (new_list[posToDelete-1][0] + new_list[posToDelete+1][0]) / 2
                resultList.append((mean,majClass0))
            majClass0 = str(max_value)
            posToDelete = i
        else:
            new_list.pop(posToDelete)
            i -= 1
            posToDelete = i
        i += 1
        
    max_value = max(class_cpt, key=class_cpt.get)
    if majClass0 != max_value:
        if majClass0 != '' :
            mean = (new_list[posToDelete-1][0] + new_list[posToDelete+1][0]) / 2
            resultList.append((mean,majClass0))
        majClass0 = str(max_value)
        posToDelete = i
    else:
        new_list.pop(posToDelete)
        i -= 1
        posToDelete = i
        resultList.append((new_list[posToDelete-1][0] ,majClass0))
        
    return resultList

In [7]:
#test discritisation function
result_of_desc=discritisation(train_df[keys[14]].values.tolist(),train_df[keys[-1]].values.tolist(),3)
print(result_of_desc)

[(1803.5, '0-10'), (1844.0, 'More than 100 Days'), (2070.5, '0-10'), (2243.5, 'More than 100 Days'), (3262.5, '0-10'), (3265.5, '31-40'), (3266.5, '0-10'), (3267.5, '31-40'), (3268.5, '0-10'), (3270.5, '31-40'), (3272.5, '0-10'), (3734.5, '31-40'), (4631.5, '11-20'), (11008.0, '21-30')]


In [8]:
#application of the descritisation for dataset
result_of_desc = []
def application_of_desc_for_dataset(feature, n ):
    result_of_desc = discritisation(train_df[feature].values.tolist(),train_df[keys[-1]].values.tolist(),n)
    for i in train_df[keys[14]].keys().tolist():
        flage = False
        for j in result_of_desc:
            if (j[0] >= train_df.loc[i,[feature]].item()):
                train_df.loc[i,[feature]]=j[1]
                flage = True
                break
        if(not flage):
            train_df.loc[i,[feature]]= "21-30"
        print(i)
        #train_df.loc[i,[feature]]=value
    return train_df

In [9]:
#application of the descritisation for value
def application_of_desc_for_value(value):
    flage = False
    for j in result_of_desc:
        if (j[0] >= value):
            output=j[1]
            flage = True
            break
    if(not flage):
        output= "21-30"
    return output

In [ ]:
application_of_desc_for_dataset(keys[14],3)

In [11]:
# save data because time issue
train_df.to_csv('raw_data.csv', index=False)

In [12]:
# deleting stay culumn from keys
keys = train_df.columns.values.tolist()
del keys[-1]

In [14]:
#get the freq of class values by eche feature value
#Compter le nombre d’apparences de chaque classe 𝐶k
def getClassFreq(feature):
    listOfDic=[]
    tt = train_desc_df.filter(items=[feature, "Stay"])
    tt.apply(pd.value_counts)
    for i in tt.groupby(feature)['Stay'].apply(list).keys():
        counts = Counter(tt.groupby(feature)['Stay'].apply(list)[i])
        #print(counts)
        dic = dict(counts)
        sorted_dic = sorted(dic.items())
        listOfDic.append((i,sorted_dic))
    return listOfDic

In [15]:
def getCFreq(feature):
    listOfDic=[]
    tt = train_desc_df.filter(items=[feature, "Stay"])
    tt.apply(pd.value_counts)
    for i in tt.groupby(feature)['Stay'].apply(list).keys():
        counts = Counter(tt.groupby(feature)['Stay'].apply(list)[i])
        #print(counts)
        dic = dict(counts)
        sorted_dic = sorted(dic.items())
        attr=[o[1] for o in sorted_dic]
        listOfDic.append(attr)
    
    return listOfDic

In [16]:
#calculate the err local function
def calculate_err_per_featuer(featuer):
    result = getCFreq(featuer)
    #print(result)
    Sum = 0
    for l in result:
        Sum= sum(l)-max(l)
    err = Sum/train_desc_df.shape[0]
    print("Erreur Par Attribut "+featuer+": " + str(err))
    return err

In [17]:
# Calculer l’erreur de chaque attribut 𝐴𝐴𝑖𝑖 (∑ 𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒 𝑉𝑉𝑗𝑗)
for i in keys:
    calculate_err_per_featuer(i)

Erreur Par Attribut Hospital_code: 0.026450600317088523
Erreur Par Attribut Hospital_type_code: 0.009739716532421902
Erreur Par Attribut City_Code_Hospital: 0.007815674370802361
Erreur Par Attribut Hospital_region_code: 0.14305233554020572
Erreur Par Attribut Available Extra Rooms in Hospital: 0.0
Erreur Par Attribut Department: 0.002848219763059984
Erreur Par Attribut Ward_Type: 2.3901144864839024e-05
Erreur Par Attribut Ward_Facility_Code: 0.25277850809053753
Erreur Par Attribut Bed Grade: 0.13008994797517467
Erreur Par Attribut City_Code_Patient: 1.593409657655935e-05
Erreur Par Attribut Type of Admission: 0.10968235378474629
Erreur Par Attribut Severity of Illness: 0.3937793286965112
Erreur Par Attribut Visitors with Patient: 0.0
Erreur Par Attribut Age: 0.0031230829290056325
Erreur Par Attribut Admission_Deposit: 0.000832556546125226


In [18]:
#deleting over fitted features
train_desc_df = train_desc_df.drop(['Available Extra Rooms in Hospital','Ward_Type','City_Code_Patient','Visitors with Patient'],axis=1)
keys = train_desc_df.columns.values.tolist()
del keys[-1]

In [19]:
# Calculer l’erreur de chaque attribut 𝐴𝐴𝑖𝑖 (∑ 𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒𝑒 𝑉𝑉𝑗𝑗)
#Sélectionner l’attribut avec l’erreur minimale
listOfErr = []
for i in keys:
    listOfErr.append(calculate_err_per_featuer(i))
print("__________________________________________________________")
print("the selected feature is "+ keys[listOfErr.index(min(listOfErr))])

Erreur Par Attribut Hospital_code: 0.026450600317088523
Erreur Par Attribut Hospital_type_code: 0.009739716532421902
Erreur Par Attribut City_Code_Hospital: 0.007815674370802361
Erreur Par Attribut Hospital_region_code: 0.14305233554020572
Erreur Par Attribut Department: 0.002848219763059984
Erreur Par Attribut Ward_Facility_Code: 0.25277850809053753
Erreur Par Attribut Bed Grade: 0.13008994797517467
Erreur Par Attribut Type of Admission: 0.10968235378474629
Erreur Par Attribut Severity of Illness: 0.3937793286965112
Erreur Par Attribut Age: 0.0031230829290056325
Erreur Par Attribut Admission_Deposit: 0.000832556546125226
__________________________________________________________
the selected feature is Admission_Deposit


In [20]:
#Construire la règle de décision
def prediction(value):
    predictionV = "21-30"
    tmp = application_of_desc_for_value(value)
    for i in getClassFreq("Admission_Deposit"):
        if tmp == i[0]:
            tmp = max(node[1] for node in i[1])
            for j in i[1]:
                if (j[1] == tmp):
                    predictionV=j[0]
    return predictionV

In [21]:
prediction(7441.0)

'21-30'

In [23]:
#accuracy 
tmp=test_df.filter(items=["Admission_Deposit", "Stay"]).values.tolist()
#tmp=train_desc_df.filter(items=["Admission_Deposit", "Stay"]).sample(n = 500).values.tolist()
cptT = 0
for i in tmp:
    if (prediction(i[0])==i[1]):
    #if (i[0]==i[1]):
        cptT += 1 
print("accuracy is : " + str(cptT/len(test_df)))

accuracy is : 0.27527525932535574
